In [2]:
from cupy import ndarray
from typing import List, Any, Tuple, Set

import cupy
import nodeleys as ndl
import sys
import copy
import tensorflow as tf

def t_update(ndl):
  pending_for_deletion = []
  for i in sys.modules:
    if i.split('.')[0] == 'nodeleys':
      pending_for_deletion.append(i)

  # print(pending_for_deletion)
    
  for i in pending_for_deletion:
    del sys.modules[i]

  import nodeleys as ndl
  return ndl

2024-09-23 15:09:32.132090: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-23 15:09:32.292490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 15:09:32.339325: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 15:09:32.352655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 15:09:32.439550: I tensorflow/core/platform/cpu_feature_guar

In [14]:
import time

ndl = t_update(ndl)
from nodeleys.system.system_func import *
from nodeleys.graph import Switch

data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,516)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,516)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(516,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(516,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

gamma0 = node_matmul(data_x, w0, 'gamma0')
y0 = node_add(gamma0, b0, 'y0')

gamma1 = node_matmul(y0, w1, 'gamma1')
y1 = node_add(gamma1, b1, 'y1')

gamma1r = node_matmul(y0, w1r, 'gamma1r')
y1rs = node_add(gamma1r, b1r, 'y1rs')

y1r = Switch(y1rs, [gamma1r], [node_mul(y1rs, 0.), node_mul(RANX, y1rs), node_mul(y1rs, 0.)], ['x<=0', 'x>0', 'x<=0'], 'y1r').compile()
# y1r = Virtual([y1r_pre], [node_mul(y1r_pre, 0.), y1r_pre], ['[0]<0', '[0]>=0'], 'y1r').compile()

gamma_rR = node_matmul(y1r, rR, 'gamma_rR')
y_rR = node_add(gamma_rR, bR, 'y_rR')

gamma_rL = node_matmul(y1r, rL, 'gamma_rL')
y_rL = node_add(gamma_rL, bL, 'y_rL')

y_rRL = node_add(y_rR, y_rL, 'y_rRL')
y_rRL = node_div(5., node_add(5., y_rRL))

merge = node_add(y1r, y_rRL, 'merge')
merge2 = node_add(merge, y1, 'merge2')

gamma2 = node_matmul(merge2, w2, 'gamma2')
y2 = node_add(gamma2, b2, 'y2')

diff = node_sub(y2, data_y, 'diff')
squared = node_pow(diff, 2., 'squared')
redsum = node_redsum(squared, 0, 'redsum')
loss = node_div(redsum, 16., 'loss')

with tf.GradientTape(persistent=True) as g:
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  tf_gamma0 = tf_data_x @ tf_w0
  tf_y0 = tf_gamma0 + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  print(id(tf_gamma1r))
  print(id(tf_gamma1rs))
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  
start_time = time.time()
grads = g.gradient(tf_loss, [tf_w0, tf_w1, tf_w2, tf_b0, tf_gamma_rR])
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
loss.adic.set_as_objective()
loss.adic.begin_backprop()
w0.get_gradient()
w1.get_gradient()
w2.get_gradient()
b0.get_gradient()
print("--- %s seconds ---" % (time.time() - start_time))

139628975913104
139628975908880
--- 0.005541801452636719 seconds ---
--- 0.0629572868347168 seconds ---


In [15]:
w0.get_gradient()

array([[  707773.59791736,  -187440.72877008,  1377239.89967088, ...,
         1475627.8634131 ,   347958.63845764, -1148056.2143788 ],
       [   96801.63119723, -1466147.14252976,  -479666.23183088, ...,
         3812930.78831207, -1606339.30127381, -3474949.90079932],
       [ -461651.55434287,  -239453.04217765,  -582191.48898274, ...,
         -167029.20975812,  -785301.922515  ,  -300355.0530448 ],
       ...,
       [  319414.81047312,  -131754.49109552,  -378746.28036418, ...,
         -642171.81518345,  -427812.00166139,   125134.49976641],
       [  721452.04218389,   -72115.74267508,   298974.7627037 , ...,
          821231.1193471 ,    82438.90969296,  -294387.840231  ],
       [-1165473.47442216,   265374.59271464,  -511304.52329182, ...,
        -1654501.83520011,  -117926.48961038,   943632.35628233]])

In [16]:
grads[0]

<tf.Tensor: shape=(1516, 516), dtype=float64, numpy=
array([[  707773.59791736,  -187440.72877008,  1377239.89967088, ...,
         1475627.8634131 ,   347958.63845764, -1148056.2143788 ],
       [   96801.63119723, -1466147.14252976,  -479666.23183088, ...,
         3812930.78831207, -1606339.30127381, -3474949.90079933],
       [ -461651.55434286,  -239453.04217765,  -582191.48898274, ...,
         -167029.20975812,  -785301.922515  ,  -300355.0530448 ],
       ...,
       [  319414.81047312,  -131754.49109552,  -378746.28036418, ...,
         -642171.81518345,  -427812.00166139,   125134.49976641],
       [  721452.04218389,   -72115.74267508,   298974.7627037 , ...,
          821231.1193471 ,    82438.90969296,  -294387.84023099],
       [-1165473.47442216,   265374.59271463,  -511304.52329182, ...,
        -1654501.83520011,  -117926.48961038,   943632.35628233]])>

In [20]:
img = ndl.Node(cupy.random.uniform(high=100, sizfe=(2, 2, 3, 3)), 'img')
flatten =  node_flatten(img, 'flatten')

In [21]:
img.tensor

array([[[[99.06203745, 78.23608785, 13.64248957],
         [19.56834423, 23.99743363, 21.34351543],
         [23.55152999, 34.97606279, 58.7469172 ]],

        [[52.91113758, 31.93808118, 52.22439406],
         [ 1.42231535, 86.50803023, 82.63915129],
         [ 0.86491391, 91.10440741, 15.32030664]]],


       [[[37.77874323, 41.40069879, 19.45345011],
         [28.38508975, 66.39952196, 67.01942814],
         [88.5943625 , 12.18776727, 95.78184851]],

        [[42.4242042 , 55.17054257, 46.10889179],
         [13.32444082,  5.58516161,  5.84209153],
         [16.9637445 , 17.36159211, 75.07280919]]]])

In [22]:
flatten.tensor

array([[99.06203745, 78.23608785, 13.64248957, 19.56834423, 23.99743363,
        21.34351543, 23.55152999, 34.97606279, 58.7469172 , 52.91113758,
        31.93808118, 52.22439406,  1.42231535, 86.50803023, 82.63915129,
         0.86491391, 91.10440741, 15.32030664],
       [37.77874323, 41.40069879, 19.45345011, 28.38508975, 66.39952196,
        67.01942814, 88.5943625 , 12.18776727, 95.78184851, 42.4242042 ,
        55.17054257, 46.10889179, 13.32444082,  5.58516161,  5.84209153,
        16.9637445 , 17.36159211, 75.07280919]])

In [6]:
w0.get_gradient()

array([[-2027956.17627339,  -849988.64668761,  -356506.17118891, ...,
         3260319.16534791,   591092.79443543, -1410751.32541803],
       [ 1831142.61690255,  3062093.2643633 ,  2232255.75784245, ...,
        -4636032.55128033,  3645691.24373744,  3163709.14838018],
       [-1388190.99962189, -1098243.08976016,  1695248.81193688, ...,
         2674277.68003849,  -907033.61930003, -1723950.96237849],
       ...,
       [ -508509.73124178,  -636294.43264531,  -174803.19636775, ...,
        -2372494.24876886, -2604942.85439146,  -738099.14980497],
       [ 2624942.08092725,  3451801.10523432,  2315442.49797524, ...,
        -5828065.09726427,  4315256.28598966,  3473637.28161869],
       [ 1898614.51283215, -1032854.05836574, -9660119.16341305, ...,
        -3251697.563462  , -5448048.2777218 ,  -288147.77448794]])

In [7]:
grads[0]

<tf.Tensor: shape=(1516, 516), dtype=float64, numpy=
array([[-2027956.17627339,  -849988.64668761,  -356506.17118891, ...,
         3260319.16534791,   591092.79443542, -1410751.32541803],
       [ 1831142.61690256,  3062093.26436329,  2232255.75784245, ...,
        -4636032.55128034,  3645691.24373744,  3163709.14838018],
       [-1388190.99962188, -1098243.08976017,  1695248.81193688, ...,
         2674277.6800385 ,  -907033.61930003, -1723950.9623785 ],
       ...,
       [ -508509.73124178,  -636294.43264531,  -174803.19636775, ...,
        -2372494.24876886, -2604942.85439146,  -738099.14980497],
       [ 2624942.08092726,  3451801.10523432,  2315442.49797524, ...,
        -5828065.09726425,  4315256.28598966,  3473637.2816187 ],
       [ 1898614.51283214, -1032854.05836573, -9660119.16341303, ...,
        -3251697.56346201, -5448048.27772179,  -288147.77448794]])>

# Explore Lab

In [18]:
tf.sqrt(784.)

<tf.Tensor: shape=(), dtype=float32, numpy=28.0>

In [8]:
data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,784*2)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,784*2)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

with tf.GradientTape(persistent=True) as g:
  tf_img = tf.Variable(tf.random.normal(shape=(1516, 2, 28, 28), dtype=tf.float64))
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  tf_flatten = tf.reshape(tf_img, shape=(tf_img.shape[0], -1))
  # tf_gamma0 = tf_data_x @ tf_w0
  tf_y0 = tf_flatten + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  

In [9]:
tf_flatten[0,900]

<tf.Tensor: shape=(), dtype=float64, numpy=-0.10608453469900318>

In [11]:
tf_img[0,1,4,4]

<tf.Tensor: shape=(), dtype=float64, numpy=-0.10608453469900318>

In [14]:
tf.reshape(g.gradient(tf_loss, tf_flatten), shape=tf_img.shape)

<tf.Tensor: shape=(1516, 2, 28, 28), dtype=float64, numpy=
array([[[[-6.93073164e+01,  1.93182075e+02, -1.91666007e+02, ...,
           4.72849761e+02,  6.88730973e+02, -2.68631847e+02],
         [-6.32845818e+01, -1.60524065e+02,  2.33486382e+02, ...,
           2.47094244e+02, -2.43957965e+02,  8.03379005e+01],
         [-1.99252685e+01, -2.81607988e+01, -4.54287911e+02, ...,
          -8.35606135e+02, -3.50729036e+01, -2.75729720e+02],
         ...,
         [-9.68097113e+01, -3.08055497e+02,  5.30993332e+02, ...,
           4.04404371e+02,  1.96815803e+02, -1.70451747e+02],
         [ 4.73525672e+02, -4.50903286e+02,  3.80326000e+02, ...,
           2.03188726e+02, -5.53511314e+01, -1.95543743e+02],
         [-8.78554571e+01, -2.78987894e+02, -1.01721961e+01, ...,
          -1.65561122e+02, -4.12100158e+02, -2.32812603e+02]],

        [[-3.97744192e+02,  2.74924324e+02,  6.40666163e+01, ...,
          -6.85428188e+01, -8.52932027e+01,  2.27238477e+02],
         [ 3.69289447e+02, -3

In [15]:
g.gradient(tf_loss, tf_img)

<tf.Tensor: shape=(1516, 2, 28, 28), dtype=float64, numpy=
array([[[[-6.93073164e+01,  1.93182075e+02, -1.91666007e+02, ...,
           4.72849761e+02,  6.88730973e+02, -2.68631847e+02],
         [-6.32845818e+01, -1.60524065e+02,  2.33486382e+02, ...,
           2.47094244e+02, -2.43957965e+02,  8.03379005e+01],
         [-1.99252685e+01, -2.81607988e+01, -4.54287911e+02, ...,
          -8.35606135e+02, -3.50729036e+01, -2.75729720e+02],
         ...,
         [-9.68097113e+01, -3.08055497e+02,  5.30993332e+02, ...,
           4.04404371e+02,  1.96815803e+02, -1.70451747e+02],
         [ 4.73525672e+02, -4.50903286e+02,  3.80326000e+02, ...,
           2.03188726e+02, -5.53511314e+01, -1.95543743e+02],
         [-8.78554571e+01, -2.78987894e+02, -1.01721961e+01, ...,
          -1.65561122e+02, -4.12100158e+02, -2.32812603e+02]],

        [[-3.97744192e+02,  2.74924324e+02,  6.40666163e+01, ...,
          -6.85428188e+01, -8.52932027e+01,  2.27238477e+02],
         [ 3.69289447e+02, -3

In [49]:
g.gradient(tf_loss, tf_img)[0,:,:,0]

<tf.Tensor: shape=(28, 28), dtype=float64, numpy=
array([[ 2.80781155e+01,  4.23474250e+02, -1.03643736e+02,
        -3.33933261e+02, -9.61520217e+01, -2.83721298e+02,
        -4.43691964e+02,  5.04108967e+02, -1.35082857e+02,
         4.55550201e+02,  2.36003017e+02,  5.25376016e+01,
        -1.60754175e+02,  3.81570503e+02,  3.69882479e+00,
         9.43926621e+01, -1.12303649e+02,  4.73018529e+02,
        -1.37730838e+02, -1.83600199e+02, -1.00198932e+02,
        -1.64809845e+02, -2.63465841e+02, -1.12460046e+02,
         2.66383276e+02,  1.60150326e+02, -4.07531656e+01,
         3.93805150e+01],
       [-2.02745862e+02,  6.02058605e+01, -7.80731937e+01,
         7.39335575e+01,  4.09099054e+01, -7.20566931e+01,
        -4.53547854e+02,  2.57210494e+02,  2.81731966e+02,
         2.12660098e+02,  6.18750768e+01,  1.42985612e+02,
         1.65113605e+02,  2.26488447e+02, -1.45379172e+02,
        -1.72655408e+02,  1.20686193e+02,  1.96973921e+02,
         3.81460304e+02,  1.24050589e+0

# Explore Lab 2

In [3]:
import time

ndl = t_update(ndl)
from nodeleys.system.system_func import *
from nodeleys.graph import Switch

data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
img = ndl.Node(cupy.random.normal(size=(1516,2,28,28)), name='img')

data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,784*2)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,784*2)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(784*2,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

flatten = node_flatten(img, 'flatten')

gamma0 = node_matmul(data_x, w0, 'gamma0')
y0 = node_add(flatten, b0, 'y0')

gamma1 = node_matmul(y0, w1, 'gamma1')
y1 = node_add(gamma1, b1, 'y1')

gamma1r = node_matmul(y0, w1r, 'gamma1r')
y1rs = node_add(gamma1r, b1r, 'y1rs')

y1r = Switch(y1rs, [gamma1r], [node_mul(y1rs, 0.), node_mul(RANX, y1rs), node_mul(y1rs, 0.)], ['x<=0', 'x>0', 'x<=0'], 'y1r').compile()
# y1r = Virtual([y1r_pre], [node_mul(y1r_pre, 0.), y1r_pre], ['[0]<0', '[0]>=0'], 'y1r').compile()

gamma_rR = node_matmul(y1r, rR, 'gamma_rR')
y_rR = node_add(gamma_rR, bR, 'y_rR')

gamma_rL = node_matmul(y1r, rL, 'gamma_rL')
y_rL = node_add(gamma_rL, bL, 'y_rL')

y_rRL = node_add(y_rR, y_rL, 'y_rRL')
y_rRL = node_div(5., node_add(5., y_rRL))

merge = node_add(y1r, y_rRL, 'merge')
merge2 = node_add(merge, y1, 'merge2')

gamma2 = node_matmul(merge2, w2, 'gamma2')
y2 = node_add(gamma2, b2, 'y2')

diff = node_sub(y2, data_y, 'diff')
squared = node_pow(diff, 2., 'squared')
redsum = node_redsum(squared, 0, 'redsum')
loss = node_div(redsum, 16., 'loss')

with tf.GradientTape(persistent=True) as g:
  tf_img = tf.Variable(tf.convert_to_tensor(img.tensor.get()))
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  # tf_gamma0 = tf_data_x @ tf_w0
  tf_flatten = tf.reshape(tf_img, shape=(tf_img.shape[0], -1))
  tf_y0 = tf_flatten + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  print(id(tf_gamma1r))
  print(id(tf_gamma1rs))
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL
  tf_y_rRL = 5./(5. + tf_y_rRL)

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1

  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  
start_time = time.time()
grads = g.gradient(tf_loss, [tf_w0, tf_w1, tf_w2, tf_b0, tf_gamma_rR])
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
loss.adic.set_as_objective()
loss.adic.begin_backprop()
w0.get_gradient()
w1.get_gradient()
w2.get_gradient()
b0.get_gradient()
print("--- %s seconds ---" % (time.time() - start_time))

140601258767952
140601258768336
--- 0.004485130310058594 seconds ---
No gradient is provided.
--- 0.013411283493041992 seconds ---


In [4]:
g.gradient(tf_loss, tf_flatten)

<tf.Tensor: shape=(1516, 1568), dtype=float64, numpy=
array([[  355.90161432,  -205.33573933,   686.76301969, ...,
          426.5506292 ,   219.57456657,   -58.84026244],
       [-2202.92048396, -7744.14303659, -5081.78021029, ...,
        -1569.33072374, -1748.24132767, -2687.79141289],
       [ 1825.3339253 , -1154.27123861,  1852.18464321, ...,
          752.15770175,  1216.11179729,  -680.95281948],
       ...,
       [   28.52933994,   -15.01458943,    43.69352366, ...,
           16.91727498,    46.28594207,   -36.62856292],
       [  -92.74966852,   222.318879  ,  -171.454901  , ...,
          -92.65980909,  -225.37545913,   277.71641551],
       [ -512.91515808,   374.51761508,  -336.66819725, ...,
         -450.11201005,  -647.07780292,   458.31268766]])>

In [5]:
flatten.get_gradient()

array([[  355.90161432,  -205.33573933,   686.76301969, ...,
          426.5506292 ,   219.57456657,   -58.84026244],
       [-2202.92048396, -7744.14303659, -5081.78021029, ...,
        -1569.33072374, -1748.24132767, -2687.79141289],
       [ 1825.3339253 , -1154.27123861,  1852.18464321, ...,
          752.15770175,  1216.11179729,  -680.95281948],
       ...,
       [   28.52933994,   -15.01458943,    43.69352366, ...,
           16.91727498,    46.28594207,   -36.62856292],
       [  -92.74966852,   222.318879  ,  -171.454901  , ...,
          -92.65980909,  -225.37545913,   277.71641551],
       [ -512.91515808,   374.51761508,  -336.66819725, ...,
         -450.11201005,  -647.07780292,   458.31268766]])

# Exploration: Numpy for Convolution

In [1]:
import numpy as np

a = np.array([[0,  1,  2,  3,  4],
              [ 5,  6,  7,  8,  9],
              [10, 11, 12, 13, 14],
              [15, 16, 17, 18, 19],
              [20, 21, 22, 23, 24]], dtype=np.float64)

sub_shape = (3,3)
view_shape = tuple(np.subtract(a.shape, sub_shape) + 1) + sub_shape
strides = a.strides + a.strides

sub_matrices = np.lib.stride_tricks.as_strided(a,view_shape,strides)

In [5]:
a = np.random.uniform(size=(20,3,6,6), high=100).astype(int)
sub_shape = (20,3,3,3)
view_shape = tuple(np.subtract(a.shape, sub_shape) + 1) + sub_shape
strides = a.strides + a.strides
sub_matrices = np.lib.stride_tricks.as_strided(a, view_shape[2:], strides[2:])

In [18]:
a.shape

(20, 3, 6, 6)

In [22]:
kernel_w = 3
stride_w = 1

w = 1 + int((a.shape[2] - kernel_w)/stride_w)


4

In [17]:
view_shape

(1, 1, 4, 4, 20, 3, 3, 3)

In [9]:
a[0]

array([[[37, 44, 72, 30, 23, 66],
        [54, 47, 33, 58, 95,  1],
        [41, 18,  8, 50,  3, 78],
        [85, 91, 97, 49, 88, 10],
        [92, 20, 40, 18, 23, 46],
        [35, 18, 21, 86,  3,  2]],

       [[ 0, 69, 10, 41, 55, 23],
        [ 8, 42, 13, 43, 27, 63],
        [95, 20, 50, 53, 96, 14],
        [38, 72, 60, 26, 31,  0],
        [46, 65, 66, 43, 31, 50],
        [43, 99, 54, 18, 87, 79]],

       [[33, 82, 94, 77, 63, 78],
        [21, 85, 24, 97, 83, 72],
        [ 1, 78,  5, 99, 27, 29],
        [15, 54,  2, 53, 85, 67],
        [33, 71,  7, 51, 13, 22],
        [59, 75, 38, 51, 56, 39]]])

In [12]:
sub_matrices.shape

(4, 4, 20, 3, 3, 3)

In [16]:
sub_matrices[1,0,0]

array([[[54, 47, 33],
        [41, 18,  8],
        [85, 91, 97]],

       [[ 8, 42, 13],
        [95, 20, 50],
        [38, 72, 60]],

       [[21, 85, 24],
        [ 1, 78,  5],
        [15, 54,  2]]])

In [3]:
view_shape

(1, 1, 4, 4, 20, 3, 3, 3)

In [4]:
strides

(864, 288, 48, 8, 864, 288, 48, 8)

In [6]:
import cupy 

batch_size = 20
channel = 3

kernel_width = 3
kernel_height = 3

domain_width = 6
domain_height = 6

stride_width = 1
stride_height = 1

block = cupy.random.uniform(size=(batch_size, channel, domain_width, domain_height), high=100).astype(int)
kernel = cupy.random.uniform(size=(5, channel, kernel_width, kernel_height), high = 10).astype(int)

new_width = 1 + int((domain_width - kernel_width)/stride_width)
new_height = 1 + int((domain_height - kernel_height)/stride_height)

pick_shape = (new_height, new_width, batch_size, channel, kernel_height, kernel_width)
memory_skip = (block.strides + block.strides)[2:]

sub_block = cupy.lib.stride_tricks.as_strided(block, pick_shape, memory_skip)

In [15]:
ndl = t_update(ndl)
from nodeleys.system.system_func import *
from nodeleys.graph import Switch

blocks = ndl.Node(block, name='blocks')
kernels = ndl.Node(kernel, name='kernels')

conv2d = node_conv2d(blocks, kernels, (1,1), 'conv2d')

In [16]:
conv2d

Node
Name:conv2d
Memory:140547776881168
is_weight:False
is_constant:False
data_shape:(20, 5, 4, 4)

In [17]:
import tensorflow as tf
tf_block = tf.convert_to_tensor(block.get(), dtype=tf.float64)
tf_kernel = tf.transpose(tf.convert_to_tensor(kernel, dtype=tf.float64), perm=[2,3,1,0])
tf.nn.conv2d(tf_block, tf_kernel, strides=[1,1,1,1], padding='VALID', data_format='NCHW')

I0000 00:00:1727081175.801140     615 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727081175.838561     615 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727081175.838591     615 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727081175.841352     615 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727081175.841379     615 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [21]:
u = np.einsum('ijklmn,rlmn', sub_block, kernel)
np.reshape(u, newshape=(u.shape[2], u.shape[3], u.shape[0], u.shape[1]))

array([[[[6291, 3558, 6658, 5803],
         [5021, 5331, 4336, 5822],
         [5088, 4694, 7451, 5040],
         [7655, 6618, 5648, 5657]],

        [[3811, 6771, 6525, 5061],
         [5844, 4623, 6432, 5891],
         [4252, 6435, 5298, 7538],
         [7016, 4766, 4912, 4053]],

        [[5635, 5850, 4231, 7750],
         [5830, 8281, 7283, 5843],
         [5701, 4305, 7260, 6945],
         [5069, 6289, 4115, 7545]],

        [[6484, 5284, 6281, 5112],
         [7524, 6280, 5068, 4880],
         [3754, 6080, 5966, 3686],
         [6191, 4734, 6990, 6774]],

        [[4777, 5586, 3725, 5976],
         [6028, 4134, 6611, 4569],
         [6811, 6203, 5357, 6464],
         [3863, 7373, 6053, 4800]]],


       [[[5847, 4192, 6504, 5709],
         [4030, 6805, 4704, 7256],
         [6660, 5310, 6663, 4307],
         [7121, 6680, 5259, 6154]],

        [[4771, 6265, 6148, 4686],
         [7582, 5434, 7264, 6413],
         [5105, 6770, 4894, 6868],
         [6096, 5404, 6797, 4852]],

    

In [10]:
view_shape

(1, 1, 4, 4, 20, 3, 3, 3)

In [4]:
a.strides

(864, 288, 48, 8)

In [20]:
from functools import reduce
import operator
sub_matrices.reshape((reduce(operator.mul, sub_matrices.shape[0:4]),) + sub_matrices.shape[4:])[4]

array([[[[55, 80, 20],
         [15, 33, 44],
         [77, 78,  1]],

        [[96, 69, 38],
         [80,  1,  9],
         [85, 19, 95]],

        [[33, 32, 21],
         [99, 57, 24],
         [92, 50, 91]]],


       [[[56, 79, 72],
         [ 2, 44,  3],
         [12, 65, 94]],

        [[69, 76, 67],
         [19, 95,  2],
         [45, 64,  4]],

        [[85, 73, 95],
         [43, 48, 24],
         [39, 42, 87]]],


       [[[73, 57,  2],
         [82,  1, 49],
         [15, 83, 51]],

        [[13,  2, 24],
         [54, 63, 78],
         [38, 21, 14]],

        [[94, 26,  5],
         [ 5, 60, 32],
         [97, 77, 87]]],


       [[[51, 94, 43],
         [14, 85,  3],
         [46, 84, 64]],

        [[41, 60,  1],
         [72, 34, 59],
         [68, 60, 45]],

        [[16,  9, 11],
         [71, 65, 71],
         [ 5, 40, 49]]],


       [[[28, 84, 42],
         [ 8, 11, 35],
         [40, 71, 67]],

        [[66, 74, 31],
         [69, 15, 97],
         [53, 89, 85]],

In [16]:
id(sub_matrices)

139666841815152

In [17]:
id(a)

139666841816688

In [9]:
sub_matrices.shape

(1, 13, 13, 3, 3, 3)

In [15]:
sub_matrices[0,0,2]

array([[[-0.34222367,  0.07832965, -0.24888853],
        [-0.68971442,  1.62167918, -0.67044116],
        [-0.59357007,  0.11655943, -1.87683302]],

       [[ 1.40873748,  0.03119758,  1.51857799],
        [ 0.13833391, -2.13709579,  0.26359758],
        [ 0.08912265,  1.15598166,  0.23029319]],

       [[-1.21138422, -0.94990054,  0.02865928],
        [-0.09158049, -0.9868813 ,  1.06084416],
        [ 0.53327552,  0.33855394, -0.22776146]]])

In [33]:
strides

(1800, 120, 8, 1800, 120, 8)

In [35]:
view_shape

(1, 13, 13, 3, 3, 3)

In [10]:
sub_shape

(3, 3, 3)

In [7]:
view_shape

(1, 13, 13, 3, 3, 3)

In [97]:
sub_matrices.reshape((9,3,3))

array([[[ 0.,  1.,  2.],
        [ 5.,  6.,  7.],
        [10., 11., 12.]],

       [[ 1.,  2.,  3.],
        [ 6.,  7.,  8.],
        [11., 12., 13.]],

       [[ 2.,  3.,  4.],
        [ 7.,  8.,  9.],
        [12., 13., 14.]],

       [[ 5.,  6.,  7.],
        [10., 11., 12.],
        [15., 16., 17.]],

       [[ 6.,  7.,  8.],
        [11., 12., 13.],
        [16., 17., 18.]],

       [[ 7.,  8.,  9.],
        [12., 13., 14.],
        [17., 18., 19.]],

       [[10., 11., 12.],
        [15., 16., 17.],
        [20., 21., 22.]],

       [[11., 12., 13.],
        [16., 17., 18.],
        [21., 22., 23.]],

       [[12., 13., 14.],
        [17., 18., 19.],
        [22., 23., 24.]]])

81

In [83]:
strides

(40, 8, 40, 8)

In [93]:
sub_matrices.shape

(3, 3, 3, 3)

In [78]:
view_shape = (2,3,3)
strides =(1*8,5*8,1*8)
sub_matrices = np.lib.stride_tricks.as_strided(a,view_shape,strides)
sub_matrices

array([[[ 0.,  1.,  2.],
        [ 5.,  6.,  7.],
        [10., 11., 12.]],

       [[ 1.,  2.,  3.],
        [ 6.,  7.,  8.],
        [11., 12., 13.]]])

In [15]:
view_shape

(3, 3, 3, 3)

In [48]:
view_shape

(2, 2, 4, 4)

In [49]:
sub_matrices

array([[[[ 0,  1,  2,  3],
         [ 5,  6,  7,  8],
         [10, 11, 12, 13],
         [15, 16, 17, 18]],

        [[ 1,  2,  3,  4],
         [ 6,  7,  8,  9],
         [11, 12, 13, 14],
         [16, 17, 18, 19]]],


       [[[ 5,  6,  7,  8],
         [10, 11, 12, 13],
         [15, 16, 17, 18],
         [20, 21, 22, 23]],

        [[ 6,  7,  8,  9],
         [11, 12, 13, 14],
         [16, 17, 18, 19],
         [21, 22, 23, 24]]]])

In [45]:
strides

(40, 8, 40, 8)

In [39]:
b = np.random.normal(size=(2,4,5))
b.strides

(160, 40, 8)

In [44]:
sub_matrices

array([[[[ 0,  1,  2,  3],
         [ 5,  6,  7,  8],
         [10, 11, 12, 13],
         [15, 16, 17, 18]],

        [[ 1,  2,  3,  4],
         [ 6,  7,  8,  9],
         [11, 12, 13, 14],
         [16, 17, 18, 19]]],


       [[[ 5,  6,  7,  8],
         [10, 11, 12, 13],
         [15, 16, 17, 18],
         [20, 21, 22, 23]],

        [[ 6,  7,  8,  9],
         [11, 12, 13, 14],
         [16, 17, 18, 19],
         [21, 22, 23, 24]]]])